In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
import os

In [3]:
import numpy as np
import pandas as pd
import tensorflow
import keras
from keras.models import Model
from keras.layers import Input, LSTM, Dense,TimeDistributed,Embedding,Bidirectional
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from string import digits
import nltk
import re
import string
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [8]:
df = pd.read_excel('/kaggle/input/mar-eng-translation-dataset/Book1.xlsx', header=None, names=["English","Marathi","Data" ])
df.head()

,English,Marathi,Data
0,Go.,जा.,CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #3138228 (sabretou)
1,Run!,पळ!,CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #3138217 (sabretou)
2,Run!,धाव!,CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #3138218 (sabretou)
3,Run!,पळा!,CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #3138219 (sabretou)
4,Run!,धावा!,CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #3138220 (sabretou)


In [9]:
# df['Data'].str.split('\t', expand=True)

In [10]:
# df['Marathi'] = df['Data'].str.split('\t', expand=True)[1]
# df['English'] = df['Data'].str.split('\t', expand=True)[0]

In [11]:
df = df.drop(['Data'], axis=1)

In [12]:
# df = df[:30000]

In [13]:
df.head()

,English,Marathi
0,Go.,जा.
1,Run!,पळ!
2,Run!,धाव!
3,Run!,पळा!
4,Run!,धावा!


In [14]:
df.shape

(39961, 2)

In [15]:
# Lowercase all characters
df['English']=df['English'].apply(lambda x: str(x))
df['Marathi']=df['Marathi'].apply(lambda x: str(x))
df['English']=df['English'].apply(lambda x: x.lower())
df['Marathi']=df['Marathi'].apply(lambda x: x.lower())

In [19]:
# Import stopwords with nltk.
import nltk
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


False

In [20]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [21]:
stop

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [22]:
df['English']=df['English'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df['Marathi']=df['Marathi'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [23]:
# Remove quotes
df['English']=df['English'].apply(lambda x: re.sub("'", '', x))
df['Marathi']=df['Marathi'].apply(lambda x: re.sub("'", '', x))

In [24]:
df.head()

,English,Marathi
0,go.,जा.
1,run!,पळ!
2,run!,धाव!
3,run!,पळा!
4,run!,धावा!


In [25]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
df['English']=df['English'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
df['Marathi']=df['Marathi'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [26]:
df.head()

,English,Marathi
0,go,जा
1,run,पळ
2,run,धाव
3,run,पळा
4,run,धावा


In [27]:
df.tail()

,English,Marathi
39956,saying like fish bones really good reason liking fish,हड्डींमुळे मासे आवडत नाही असं म्हणणं हे काय मासे नावडण्यासाठी चांगलं कारण नाहीये
39957,japanese parliament today officially elected ryoutarou hashimoto countrys 52nd prime minister,आज जपानी संसदेने अधिकृतरित्या र्‍यौतारौ हाशिमोतो यांना देशाचे ५२ावे पंतप्रधान म्हणून निवडले
39958,tom tried sell old vcr instead throwing away one would buy it ended throwing away,टॉमने त्याचा जुना व्हीसीआर फेकून टाकण्याऐवजी विकून टाकण्याचा प्रयत्न केला पण विकत घ्यायला कोणीच तयार नव्हतं म्हणून त्याला तो शेवटी फेकूनच टाकायला लागला
39959,cant view flash content ipad however easily email urls web pages view content regular computer get home,आयपॅडवर फ्लॅश आशय बघता येत नाही पण तुम्ही त्या वेब पानांचे यूआरएल स्वतःला ईमेल करून तोच आशय घरी पोहोचल्यावर आपल्या रोजच्या संगणकावर पाहू शकता
39960,1969 roger miller recorded song called you want love today song better known in summer time first song wrote sang became popular,१९६९मध्ये रॉजर मिलरने यू डोन्ट वॉन्ट माय लव्ह नावाचं गाणं रेकॉर्ड केलं आज हे गाणं इन द समर टाइम म्हणून जास्त प्रसिद्ध आहे त्याने लिहिलेलं व गायलेलं हे असं पहिलं गाणं होतं जे लोकप्रिय झालं


In [28]:
remove_digits = str.maketrans('', '', digits)

In [29]:
remove_digits

{48: None,
 49: None,
 50: None,
 51: None,
 52: None,
 53: None,
 54: None,
 55: None,
 56: None,
 57: None}

In [30]:
a = df['English'][0].translate(remove_digits)

In [31]:
a

'go'

In [32]:
a.strip()

'go'

In [33]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
df['English']=df['English'].apply(lambda x: x.translate(remove_digits))
df['Marathi']=df['Marathi'].apply(lambda x: x.translate(remove_digits))

df['Marathi'] = df['Marathi'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

In [34]:
# Remove extra spaces
df['English']=df['English'].apply(lambda x: x.strip())
df['Marathi']=df['Marathi'].apply(lambda x: x.strip())
df['English']=df['English'].apply(lambda x: re.sub(" +", " ", x))
df['Marathi']=df['Marathi'].apply(lambda x: re.sub(" +", " ", x))

In [35]:
# Add start and end tokens to target sequences
df['English'] = df['English'].apply(lambda x : 'START_ '+ x + ' _END')

In [36]:
df['English'][0]

'START_ go _END'

In [37]:
### Get English Vocabulary
all_eng_words=set()
for eng in df['English']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

In [38]:
all_eng_words

{'colonists',
 'ethiopian',
 'lansing',
 'drugs',
 'have',
 'dj',
 'single',
 'john',
 'posted',
 'ballpoint',
 'kosher',
 'want',
 'raincoat',
 'poker',
 'banks',
 'rodicas',
 'letters',
 'fighter',
 'daughters',
 'scream',
 'beg',
 'solar',
 'lilienthal',
 'surprised',
 'express',
 'samsung',
 'undress',
 'mosquito',
 'population',
 'fortyfour',
 'fit',
 'head',
 'shout',
 'muffins',
 'spot',
 'gi',
 'insects',
 'sign',
 'breaks',
 'accidentally',
 'wiped',
 'confirm',
 'lifeboats',
 'confessed',
 'pieces',
 'brunch',
 'seldom',
 'easily',
 'grenade',
 'leal',
 'reasons',
 'pianist',
 'orders',
 'fairly',
 'talked',
 'yet',
 'life',
 'bottle',
 'cupcake',
 'maharashtra',
 'towns',
 'report',
 'mysteries',
 'miles',
 'count',
 'hacked',
 'belongs',
 'peaceful',
 'gummy',
 'sauna',
 'quran',
 'check',
 'illinois',
 'hayes',
 'road',
 'protest',
 'key',
 'subject',
 'typed',
 'places',
 'francos',
 'european',
 'bothers',
 'tokyo',
 'shillings',
 'racket',
 'desktop',
 'off',
 'lunar',


In [39]:
### Get Marathi Vocabulary
all_marathi_words=set()
for mar in df['Marathi']:
    for word in mar.split():
        if word not in all_marathi_words:
            all_marathi_words.add(word)

In [40]:
# li = list(all_marathi_words)

In [41]:
# li

In [42]:
# li.sort()

In [43]:
# li.pop(0)

In [44]:
# li

In [45]:
# all_marathi_words=set(li)

In [46]:
# all_marathi_words

In [47]:
df['length_eng']=df['English'].apply(lambda x:len(x.split(" ")))
df['length_mar']=df['Marathi'].apply(lambda x:len(x.split(" ")))

In [48]:
df['length_eng']

0        3 
1        3 
2        3 
3        3 
4        3 
        .. 
39956    11
39957    13
39958    17
39959    19
39960    23
Name: length_eng, Length: 39961, dtype: int64

In [49]:
df.head()

,English,Marathi,length_eng,length_mar
0,START_ go _END,जा,3,1
1,START_ run _END,पळ,3,1
2,START_ run _END,धाव,3,1
3,START_ run _END,पळा,3,1
4,START_ run _END,धावा,3,1


In [50]:
df['length_mar']

0        1 
1        1 
2        1 
3        1 
4        1 
        .. 
39956    13
39957    12
39958    23
39959    24
39960    35
Name: length_mar, Length: 39961, dtype: int64

In [51]:
df[df['length_eng']>30].shape

(0, 4)

In [52]:
df[df['length_mar']>30].shape

(1, 4)

In [53]:
df=df[df['length_eng']<=20]
df=df[df['length_mar']<=20]

In [54]:
print("maximum length of Marathi Sentence ",max(df['length_mar']))
print("maximum length of English Sentence ",max(df['length_eng']))

maximum length of Marathi Sentence  19
maximum length of English Sentence  13


In [55]:
max_length_src=max(df['length_mar'])
max_length_tar=max(df['length_eng'])

In [56]:
print(max_length_src)

19


In [57]:
print(max_length_tar)

13


In [58]:
input_words = sorted(list(all_marathi_words))
target_words = sorted(list(all_eng_words))
num_encoder_tokens = len(all_marathi_words)
num_decoder_tokens = len(all_eng_words)
num_encoder_tokens, num_decoder_tokens

(13608, 5624)

In [59]:
input_words

['b',
 'ds',
 'h₂o',
 'ntt',
 'tatoebaorg',
 'uk',
 'अ',
 'अँकरेजमार्गे',
 'अँजिलीस',
 'अँटीव्हायरस',
 'अँड',
 'अँडी',
 'अँडीजमधल्या',
 'अँब्युलन्स',
 'अंक',
 'अंकांनी',
 'अंकारा',
 'अंकारात',
 'अंकारामध्ये',
 'अंग',
 'अंगठ्या',
 'अंगाला',
 'अंगोला',
 'अंघोळ',
 'अंठ्यांपेक्षा',
 'अंड',
 'अंडरवुडचा',
 'अंडरवेअर',
 'अंडी',
 'अंडींचा',
 'अंडीसुद्धा',
 'अंड्यांवर',
 'अंत',
 'अंतर',
 'अंतराळयात्री',
 'अंतराळयान',
 'अंतराळयानाचं',
 'अंतराळयानातून',
 'अंतराळवीर',
 'अंतराळात',
 'अंतरित',
 'अंतिम',
 'अंदाज',
 'अंदाजे',
 'अंधार',
 'अंधाराची',
 'अंधारात',
 'अंमलात',
 'अंशांकित',
 'अकरा',
 'अकरावीत',
 'अकल्पनीय',
 'अकाउंट',
 'अकाउंटमध्ये',
 'अकॉर्डियन',
 'अक्कलदाढ',
 'अक्वॅरियममध्ये',
 'अक्षर',
 'अक्षरं',
 'अक्षरांनी',
 'अक्षरांमध्ये',
 'अक्षरे',
 'अखाद्य',
 'अखिलेउस',
 'अख्खं',
 'अख्खा',
 'अख्ख्या',
 'अगं',
 'अगदी',
 'अगोदर',
 'अगोदरच',
 'अचानक',
 'अचानकच',
 'अचूक',
 'अच्छा',
 'अजिबात',
 'अजिबातच',
 'अजीबात',
 'अजुनपर्यंत',
 'अजुनही',
 'अजून',
 'अजूनच',
 'अजूनपर्यंत',
 'अजूनही',
 'अज्ञात',
 'अट',

In [60]:
target_words

['START_',
 '_END',
 'a',
 'abbreviation',
 'ability',
 'able',
 'about',
 'above',
 'abroad',
 'absent',
 'absolute',
 'absolutely',
 'accent',
 'accept',
 'acceptable',
 'accepted',
 'access',
 'accessible',
 'accident',
 'accidentally',
 'accomplices',
 'according',
 'accordion',
 'account',
 'accused',
 'accusing',
 'accustomed',
 'ace',
 'aces',
 'ache',
 'aches',
 'achille',
 'achilles',
 'acid',
 'acknowledged',
 'acquired',
 'acres',
 'across',
 'act',
 'acted',
 'acting',
 'action',
 'activist',
 'actor',
 'actors',
 'actress',
 'acts',
 'actual',
 'actually',
 'add',
 'added',
 'addict',
 'addicted',
 'addition',
 'additional',
 'addon',
 'address',
 'administrative',
 'admire',
 'admit',
 'admitted',
 'adopt',
 'adopted',
 'ads',
 'adultery',
 'advance',
 'advantage',
 'adverb',
 'advertisements',
 'advice',
 'advised',
 'adviser',
 'advisers',
 'advisor',
 'affair',
 'affairs',
 'afford',
 'afghanistan',
 'afraid',
 'africa',
 'african',
 'africans',
 'after',
 'afternoon',

In [61]:
num_decoder_tokens

5624

In [62]:
num_decoder_tokens += 1

In [63]:
num_decoder_tokens

5625

In [64]:
num_encoder_tokens

13608

In [65]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [66]:
input_token_index

{'b': 1,
 'ds': 2,
 'h₂o': 3,
 'ntt': 4,
 'tatoebaorg': 5,
 'uk': 6,
 'अ': 7,
 'अँकरेजमार्गे': 8,
 'अँजिलीस': 9,
 'अँटीव्हायरस': 10,
 'अँड': 11,
 'अँडी': 12,
 'अँडीजमधल्या': 13,
 'अँब्युलन्स': 14,
 'अंक': 15,
 'अंकांनी': 16,
 'अंकारा': 17,
 'अंकारात': 18,
 'अंकारामध्ये': 19,
 'अंग': 20,
 'अंगठ्या': 21,
 'अंगाला': 22,
 'अंगोला': 23,
 'अंघोळ': 24,
 'अंठ्यांपेक्षा': 25,
 'अंड': 26,
 'अंडरवुडचा': 27,
 'अंडरवेअर': 28,
 'अंडी': 29,
 'अंडींचा': 30,
 'अंडीसुद्धा': 31,
 'अंड्यांवर': 32,
 'अंत': 33,
 'अंतर': 34,
 'अंतराळयात्री': 35,
 'अंतराळयान': 36,
 'अंतराळयानाचं': 37,
 'अंतराळयानातून': 38,
 'अंतराळवीर': 39,
 'अंतराळात': 40,
 'अंतरित': 41,
 'अंतिम': 42,
 'अंदाज': 43,
 'अंदाजे': 44,
 'अंधार': 45,
 'अंधाराची': 46,
 'अंधारात': 47,
 'अंमलात': 48,
 'अंशांकित': 49,
 'अकरा': 50,
 'अकरावीत': 51,
 'अकल्पनीय': 52,
 'अकाउंट': 53,
 'अकाउंटमध्ये': 54,
 'अकॉर्डियन': 55,
 'अक्कलदाढ': 56,
 'अक्वॅरियममध्ये': 57,
 'अक्षर': 58,
 'अक्षरं': 59,
 'अक्षरांनी': 60,
 'अक्षरांमध्ये': 61,
 'अक्षरे': 62,
 'अखाद्य': 63,
 

In [67]:
target_token_index

{'START_': 1,
 '_END': 2,
 'a': 3,
 'abbreviation': 4,
 'ability': 5,
 'able': 6,
 'about': 7,
 'above': 8,
 'abroad': 9,
 'absent': 10,
 'absolute': 11,
 'absolutely': 12,
 'accent': 13,
 'accept': 14,
 'acceptable': 15,
 'accepted': 16,
 'access': 17,
 'accessible': 18,
 'accident': 19,
 'accidentally': 20,
 'accomplices': 21,
 'according': 22,
 'accordion': 23,
 'account': 24,
 'accused': 25,
 'accusing': 26,
 'accustomed': 27,
 'ace': 28,
 'aces': 29,
 'ache': 30,
 'aches': 31,
 'achille': 32,
 'achilles': 33,
 'acid': 34,
 'acknowledged': 35,
 'acquired': 36,
 'acres': 37,
 'across': 38,
 'act': 39,
 'acted': 40,
 'acting': 41,
 'action': 42,
 'activist': 43,
 'actor': 44,
 'actors': 45,
 'actress': 46,
 'acts': 47,
 'actual': 48,
 'actually': 49,
 'add': 50,
 'added': 51,
 'addict': 52,
 'addicted': 53,
 'addition': 54,
 'additional': 55,
 'addon': 56,
 'address': 57,
 'administrative': 58,
 'admire': 59,
 'admit': 60,
 'admitted': 61,
 'adopt': 62,
 'adopted': 63,
 'ads': 64,
 '

In [68]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [69]:
reverse_input_char_index

{1: 'b',
 2: 'ds',
 3: 'h₂o',
 4: 'ntt',
 5: 'tatoebaorg',
 6: 'uk',
 7: 'अ',
 8: 'अँकरेजमार्गे',
 9: 'अँजिलीस',
 10: 'अँटीव्हायरस',
 11: 'अँड',
 12: 'अँडी',
 13: 'अँडीजमधल्या',
 14: 'अँब्युलन्स',
 15: 'अंक',
 16: 'अंकांनी',
 17: 'अंकारा',
 18: 'अंकारात',
 19: 'अंकारामध्ये',
 20: 'अंग',
 21: 'अंगठ्या',
 22: 'अंगाला',
 23: 'अंगोला',
 24: 'अंघोळ',
 25: 'अंठ्यांपेक्षा',
 26: 'अंड',
 27: 'अंडरवुडचा',
 28: 'अंडरवेअर',
 29: 'अंडी',
 30: 'अंडींचा',
 31: 'अंडीसुद्धा',
 32: 'अंड्यांवर',
 33: 'अंत',
 34: 'अंतर',
 35: 'अंतराळयात्री',
 36: 'अंतराळयान',
 37: 'अंतराळयानाचं',
 38: 'अंतराळयानातून',
 39: 'अंतराळवीर',
 40: 'अंतराळात',
 41: 'अंतरित',
 42: 'अंतिम',
 43: 'अंदाज',
 44: 'अंदाजे',
 45: 'अंधार',
 46: 'अंधाराची',
 47: 'अंधारात',
 48: 'अंमलात',
 49: 'अंशांकित',
 50: 'अकरा',
 51: 'अकरावीत',
 52: 'अकल्पनीय',
 53: 'अकाउंट',
 54: 'अकाउंटमध्ये',
 55: 'अकॉर्डियन',
 56: 'अक्कलदाढ',
 57: 'अक्वॅरियममध्ये',
 58: 'अक्षर',
 59: 'अक्षरं',
 60: 'अक्षरांनी',
 61: 'अक्षरांमध्ये',
 62: 'अक्षरे',
 63: 'अखाद्य',
 

In [70]:
reverse_target_char_index

{1: 'START_',
 2: '_END',
 3: 'a',
 4: 'abbreviation',
 5: 'ability',
 6: 'able',
 7: 'about',
 8: 'above',
 9: 'abroad',
 10: 'absent',
 11: 'absolute',
 12: 'absolutely',
 13: 'accent',
 14: 'accept',
 15: 'acceptable',
 16: 'accepted',
 17: 'access',
 18: 'accessible',
 19: 'accident',
 20: 'accidentally',
 21: 'accomplices',
 22: 'according',
 23: 'accordion',
 24: 'account',
 25: 'accused',
 26: 'accusing',
 27: 'accustomed',
 28: 'ace',
 29: 'aces',
 30: 'ache',
 31: 'aches',
 32: 'achille',
 33: 'achilles',
 34: 'acid',
 35: 'acknowledged',
 36: 'acquired',
 37: 'acres',
 38: 'across',
 39: 'act',
 40: 'acted',
 41: 'acting',
 42: 'action',
 43: 'activist',
 44: 'actor',
 45: 'actors',
 46: 'actress',
 47: 'acts',
 48: 'actual',
 49: 'actually',
 50: 'add',
 51: 'added',
 52: 'addict',
 53: 'addicted',
 54: 'addition',
 55: 'additional',
 56: 'addon',
 57: 'address',
 58: 'administrative',
 59: 'admire',
 60: 'admit',
 61: 'admitted',
 62: 'adopt',
 63: 'adopted',
 64: 'ads',
 6

In [71]:
df.head(15)

,English,Marathi,length_eng,length_mar
0,START_ go _END,जा,3,1
1,START_ run _END,पळ,3,1
2,START_ run _END,धाव,3,1
3,START_ run _END,पळा,3,1
4,START_ run _END,धावा,3,1
5,START_ who _END,कोण,3,1
6,START_ wow _END,वाह,3,1
7,START_ fire _END,आग,3,1
8,START_ fire _END,फायर,3,1
9,START_ help _END,वाचवा,3,1


In [72]:
from sklearn.model_selection import train_test_split
X, y = df['Marathi'], df['English']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((31966,), (7992,))

In [73]:
X_train

10178    त्याने आपली बॅग भरली                                     
7752     आता गायला सुरुवात कर                                     
5986     मी टॉमवर नजर ठेवतोय                                      
14697    बियर कोणी आणली                                           
27272    त्याने काय केलं आहे हे तुम्हाला माहीत आहे का             
                             ...                                  
6265     कुत्रा मेलेला                                            
11284    मी खेळ जिंकलेय                                           
38158    त्याच्या आजोबांनी त्याच्यासाठी एक महागडं खेळणं विकत घेतलं
860      त्याच्याकडे वाईन आहे                                     
15795    शनिवारची रात्र होती                                      
Name: Marathi, Length: 31966, dtype: object

In [74]:
y_train

10178    START_ tom packed bag _END                  
7752     START_ start singing _END                   
5986     START_ im watching tom _END                 
14697    START_ brought beer _END                    
27272    START_ know done _END                       
                 ...                                 
6265     START_ dog dead _END                        
11284    START_ game _END                            
38158    START_ grandfather bought expensive toy _END
860      START_ wine _END                            
15795    START_ saturday night _END                  
Name: English, Length: 31966, dtype: object

In [75]:
encoder_input_data = np.zeros((2, max_length_src),dtype='float32')
decoder_input_data = np.zeros((2, max_length_tar),dtype='float32')
decoder_target_data = np.zeros((2, max_length_tar, num_decoder_tokens),dtype='float32')

In [76]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [77]:
latent_dim = 300
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens+1, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [78]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens+1, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [79]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [80]:
model.summary()
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 64
epochs = 10

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 300)    4082700     ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 300)    1687800     ['input_2[0][0]']                
                                                                                              

In [81]:
model.save('mar-to-eng.h5')

In [82]:
a, b = next(generate_batch())

In [83]:
a

[array([[ 5608.,   670.,  8473., ...,     0.,     0.,     0.],
        [  623.,  3058., 12838., ...,     0.,     0.,     0.],
        [ 9619.,  4783.,  6244., ...,     0.,     0.,     0.],
        ...,
        [ 5513.,  4019., 12499., ...,     0.,     0.,     0.],
        [ 5483.,  9909.,  4532., ...,     0.,     0.,     0.],
        [ 5543.,  9672.,  6401., ...,     0.,     0.,     0.]],
       dtype=float32),
 array([[1.000e+00, 5.040e+03, 3.524e+03, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        [1.000e+00, 4.655e+03, 4.442e+03, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        [1.000e+00, 2.473e+03, 5.390e+03, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        ...,
        [1.000e+00, 3.442e+03, 2.171e+03, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        [1.000e+00, 2.185e+03, 5.249e+03, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        [1.000e+00, 4.568e+03, 3.265e+03, ..., 0.000e+00, 0.000e+00,
         0.000e+00]], dtype=float32)]

In [84]:
b

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [85]:
X_train[100]

'मी पण'

In [86]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples/batch_size,
                    epochs=10,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples/batch_size)

Epoch 1/10


/tmp/ipykernel_29/1093200829.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),


499/499 [==============================] - 66s 99ms/step - loss: 5.2393 - accuracy: 0.3044 - val_loss: 4.7626 - val_accuracy: 0.3232
Epoch 2/10
499/499 [==============================] - 24s 47ms/step - loss: 2.5545 - accuracy: 0.5352 - val_loss: 2.7924 - val_accuracy: 0.5419
Epoch 5/10
499/499 [==============================] - 23s 46ms/step - loss: 1.8638 - accuracy: 0.6368 - val_loss: 2.4414 - val_accuracy: 0.6004
Epoch 6/10
499/499 [==============================] - 23s 45ms/step - loss: 1.3405 - accuracy: 0.7267 - val_loss: 2.2193 - val_accuracy: 0.6424
Epoch 7/10
499/499 [==============================] - 23s 47ms/step - loss: 0.9539 - accuracy: 0.8018 - val_loss: 2.0597 - val_accuracy: 0.6740
Epoch 8/10
499/499 [==============================] - 21s 43ms/step - loss: 0.6739 - accuracy: 0.8641 - val_loss: 1.9694 - val_accuracy: 0.6966
Epoch 9/10
499/499 [==============================] - 23s 46ms/step - loss: 0.4733 - accuracy: 0.9086 - val_loss: 1.9152 - val_accuracy: 0.7116
Epo

In [87]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [88]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [89]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [90]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Marathi sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 21ms/step
Input Marathi sentence: त्याने आपली बॅग भरली
Actual English Translation:  tom packed bag 
Predicted English Translation:  tom packed bag 


In [91]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Marathi sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 22ms/step
Input Marathi sentence: आता गायला सुरुवात कर
Actual English Translation:  start singing 
Predicted English Translation:  start singing 


In [92]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Marathi sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 22ms/step
Input Marathi sentence: मी टॉमवर नजर ठेवतोय
Actual English Translation:  im watching tom 
Predicted English Translation:  im watching tom 


In [94]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Marathi sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 22ms/step
Input Marathi sentence: त्याने काय केलं आहे हे तुम्हाला माहीत आहे का
Actual English Translation:  know done 
Predicted English Translation:  know did 


In [95]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Marathi sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 20ms/step
Input Marathi sentence: मी टीव्ही बघत नाही
Actual English Translation:  watch tv 
Predicted English Translation:  watch tv 


In [96]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Marathi sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 22ms/step
Input Marathi sentence: तुम्ही आपल्या घरी सुखी आहात का
Actual English Translation:  happy house 
Predicted English Translation:  happy house 


In [97]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Marathi sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 24ms/step
Input Marathi sentence: मी दर सकाळी साउअरक्राउट खाते
Actual English Translation:  eat sauerkraut every morning 
Predicted English Translation:  eat sauerkraut every morning 


In [98]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Marathi sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 22ms/step
Input Marathi sentence: काय कळून आलं
Actual English Translation:  learn 
Predicted English Translation:  learn 


In [99]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Marathi sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 23ms/step
Input Marathi sentence: नवीन वर्षाच्या हार्दिक शुभेच्छा
Actual English Translation:  happy new year 
Predicted English Translation:  happy new year 


In [107]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Marathi sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 22ms/step
Input Marathi sentence: मी जवळजवळ प्रत्येक रात्री घरी असतो
Actual English Translation:  im home almost every night 
Predicted English Translation:  im home almost every night 


In [113]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Marathi sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 24ms/step
Input Marathi sentence: मी हा पिक्चर आधीच पाहिला आहे
Actual English Translation:  ive seen movie already 
Predicted English Translation:  ive seen movie already 


In [118]:
import warnings
warnings.filterwarnings("ignore")

In [119]:
from nltk.translate.bleu_score import sentence_bleu
ref = [
    "ive seen movie already".split(),
    "im home almost every night".split(),
    "eat sauerkraut every morning".split(),
    "know done".split()
]
test = "know did".split()
print('BLEU score for test-> {}'.format(sentence_bleu(ref, test)))

BLEU score for test-> 0.8408964152537145
